In [2]:
import pandas as pd

## Read movies data and store in movies table in database

In [ ]:
df = pd.read_json("/home/sanustha/Public/recommender-system/api/data/genome_2021/movie_dataset_public_final/raw/metadata.json", lines=True)

In [27]:
df.columns

Index(['title', 'directedBy', 'starring', 'dateAdded', 'avgRating', 'imdbId',
       'item_id'],
      dtype='object')

In [19]:
len(data)

84661

In [3]:
from sqlalchemy import create_engine

DB_USER="rec_user"
DB_PASSWORD="admin1234"
DB_HOST="localhost"
DB_NAME="recommendation_system"
DB_PORT="5432"

# Create the connection URL
db_url = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

# Create the SQLAlchemy engine
engine = create_engine(db_url)

# Test the connection
# try:
#     with engine.connect() as connection:
#         result = connection.execute("SELECT version();")
#         print(f"Connected to: {result.fetchone()[0]}")
# except Exception as e:
#     print(f"An error occurred: {e}")


## Load movie data in movies table

In [28]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from io import StringIO
from datetime import datetime
import os

# Database credentials
username="rec_user"
password="admin1234"
host="localhost"
database="recommendation_system"
port="5432"



# Map and process the DataFrame
df_mapped = pd.DataFrame({
    'movie_id': df['item_id'],
    'title': df['title'],
    'genre': df['starring'],
    'director': df['directedBy'],
    'duration_min': None,
    'poster': None
})

# Extract 'release_year' from 'dateAdded'
def extract_year(date_str):
    if pd.isnull(date_str):
        return pd.NA  # Use pandas' NA value for missing data
    try:
        return int(datetime.strptime(date_str[:10], '%Y-%m-%d').year)
    except ValueError:
        return pd.NA

# Apply the function and convert to Int64
df_mapped['release_year'] = df['dateAdded'].apply(extract_year).astype('Int64')

# Verify data types
print(df_mapped.dtypes)

# Convert DataFrame to CSV buffer
csv_buffer = StringIO()
df_mapped.to_csv(csv_buffer, index=False, header=False, sep='\t', na_rep='\\N')
csv_buffer.seek(0)

# Use psycopg2 to connect
conn = psycopg2.connect(
    dbname=database,
    user=username,
    password=password,
    host=host,
    port=port
)
cursor = conn.cursor()

# Define the columns
columns = ', '.join(df_mapped.columns)

# Execute the COPY command
copy_query = f"COPY movies ({columns}) FROM STDIN WITH (FORMAT CSV, DELIMITER '\t', NULL '\\N')"
cursor.copy_expert(sql=copy_query, file=csv_buffer)

# Commit and close
conn.commit()
cursor.close()
conn.close()

print("Data insertion completed successfully using COPY command.")

movie_id         int64
title           object
genre           object
director        object
duration_min    object
poster          object
release_year     Int64
dtype: object
Data insertion completed successfully using COPY command.


## Query Movie table to see if the data has been loaded

In [8]:
df = pd.read_sql_query("SELECT setval('movies_movie_id_seq', COALESCE((SELECT MAX(movie_id) + 1 FROM movies), 1), false);", engine)

In [9]:
df.head()

,setval
0,239315


In [23]:
df = pd.read_sql_query("SELECT * FROM users;", engine)

In [24]:
df.head()

,user_id,user_name,email,password,role,created_at,updated_at
0,1,nikstha,nikshres522@gmail.com,$2b$12$VMO9/sWadYPbhZbvwW5HFeqLqJIOGaZTXwQAl9N...,user,2024-10-20 15:43:38.339976-04:00,NaT
1,2,Nikita Shrestha,test@gmail.com,test123,user,2024-10-20 16:02:01.931910-04:00,NaT
2,3,rpralesh,pralesh@gmail.com,test123,rec_user,2024-10-20 16:02:01.931910-04:00,2024-10-20 20:02:01.931910+00:00
3,4,Pralesh Rayamajhi,pralesh1@gmail.com,$2b$12$hM4D81Pw5.8YbSXfI2qeTOsdBjWN8PlpGJGmOWv...,user,2024-10-31 10:28:27.988607-04:00,NaT
4,5,nikstha,nikshres52@gmail.com,$2b$12$xrQ602KlJ.4PoWK0Idi2C.8rwe8DYQQxzXbroKj...,user,2024-11-07 20:22:48.435697-05:00,NaT


In [26]:
df = pd.read_sql_query("SELECT * FROM ratings;", engine)
df.head()

,id,user_id,movie_id,rating,created_at,updated_at
0,2,6,1,5.0,2024-11-10 15:46:55.279250+00:00,None
1,3,6,6,3.0,2024-11-10 15:46:55.279250+00:00,None
2,4,6,10,3.0,2024-11-10 15:46:55.279250+00:00,None
3,5,6,16,3.0,2024-11-10 15:46:55.279250+00:00,None
4,6,6,17,3.0,2024-11-10 15:46:55.279250+00:00,None


# Load user data (Creating dummy users)

In [3]:
import pandas as pd

ratings = pd.read_json("/home/sanustha/Public/recommender-system/api/data/genome_2021/movie_dataset_public_final/raw/filtered_ratings.json", lines=True)

In [12]:
import pandas as pd
import requests

# Step 1: Read the JSON File into a DataFrame
df = pd.read_json("/home/sanustha/Public/recommender-system/api/data/genome_2021/movie_dataset_public_final/raw/filtered_ratings.json", orient='records', lines=True)
# Step 2: Extract Unique User IDs from the DataFrame
user_ids = df['user_id'].unique()

# Step 3: Define the API Endpoints and Headers
API_SIGNUP_URL = 'http://localhost:8000/signup'  # Replace with your actual signup API endpoint
API_USERS_URL = 'http://localhost:8000/users/'    # Endpoint to get all users
HEADERS = {
    'Content-Type': 'application/json',
    # 'Authorization': 'Bearer YOUR_API_TOKEN',  # Uncomment and set if your API requires authentication
}

# Step 4: Retrieve Existing Users from the Database
try:
    # Send a GET request to retrieve all existing users
    response = requests.get(API_USERS_URL, headers=HEADERS)
    if response.status_code == 200:
        existing_users = response.json()
        # Extract existing user IDs into a set for fast lookup
        existing_user_ids = {user['user_id'] for user in existing_users}
        print(f"Retrieved {len(existing_user_ids)} existing users from the database.")
    else:
        print(f"Failed to retrieve existing users. Status Code: {response.status_code}, Message: {response.text}")
        existing_user_ids = set()
except requests.exceptions.RequestException as e:
    print(f"An error occurred while retrieving existing users: {e}")
    existing_user_ids = set()

# Step 5: Iterate Over User IDs and Create Users If They Don't Exist
for user_id in user_ids:
    if int(user_id) in existing_user_ids:
        print(f"User with ID {user_id} already exists. Skipping.")
        continue  # Skip to the next user_id

    # Prepare the payload with dummy data
    payload = {
        'user_name': f'User{user_id}',
        'email': f'user{user_id}@example.com',
        'password': 'dummy_password',  # In production, ensure secure password handling
    }

    try:
        # Send a POST request to the signup API endpoint
        response = requests.post(API_SIGNUP_URL, json=payload, headers=HEADERS)

        if response.status_code == 200:
            # User created successfully
            print(f"User with ID {user_id} created successfully.")
        elif response.status_code == 409:
            # User already exists (assuming 409 Conflict for existing users)
            print(f"User with ID {user_id} already exists according to API. Skipping.")
        else:
            # Handle other possible responses
            print(f"Failed to create user with ID {user_id}. Status Code: {response.status_code}, Message: {response.text}")

    except requests.exceptions.RequestException as e:
        # Handle any request exceptions
        print(f"An error occurred while creating user with ID {user_id}: {e}")


Retrieved 8 existing users from the database.
Failed to create user with ID 19. Status Code: 400, Message: {"detail":"Email already registered"}
Failed to create user with ID 77. Status Code: 400, Message: {"detail":"Email already registered"}
Failed to create user with ID 75. Status Code: 400, Message: {"detail":"Email already registered"}


In [10]:
# Step 2: Extract Unique User IDs from the DataFrame
user_ids = df['user_id'].unique()

# Step 3: Define the API Endpoints and Headers
API_SIGNUP_URL = 'http://localhost:8000/signup'  # Replace with your actual signup API endpoint
API_USERS_URL = 'http://localhost:8000/users/'    # Endpoint to get all users
HEADERS = {
    'Content-Type': 'application/json',
    # 'Authorization': 'Bearer YOUR_API_TOKEN',  # Uncomment and set if your API requires authentication
}

# Step 4: Retrieve Existing Users from the Database
try:
    # Send a GET request to retrieve all existing users
    response = requests.get(API_USERS_URL, headers=HEADERS)
    if response.status_code == 200:
        existing_users = response.json()
        # Extract existing user IDs into a set for fast lookup
        existing_user_ids = {user['user_id'] for user in existing_users}
        print(f"Retrieved {len(existing_user_ids)} existing users from the database.")
    else:
        print(f"Failed to retrieve existing users. Status Code: {response.status_code}, Message: {response.text}")
        existing_user_ids = set()
except requests.exceptions.RequestException as e:
    print(f"An error occurred while retrieving existing users: {e}")
    existing_user_ids = set()

Retrieved 8 existing users from the database.


In [14]:
existing_users

[{'user_id': 1,
  'user_name': 'nikstha',
  'email': 'nikshres522@gmail.com',
  'role': 'user',
  'created_at': '2024-10-20T15:43:38.339976-04:00',
  'updated_at': None},
 {'user_id': 2,
  'user_name': 'Nikita Shrestha',
  'email': 'test@gmail.com',
  'role': 'user',
  'created_at': '2024-10-20T16:02:01.931910-04:00',
  'updated_at': None},
 {'user_id': 3,
  'user_name': 'rpralesh',
  'email': 'pralesh@gmail.com',
  'role': 'rec_user',
  'created_at': '2024-10-20T16:02:01.931910-04:00',
  'updated_at': '2024-10-20T16:02:01.931910-04:00'},
 {'user_id': 4,
  'user_name': 'Pralesh Rayamajhi',
  'email': 'pralesh1@gmail.com',
  'role': 'user',
  'created_at': '2024-10-31T10:28:27.988607-04:00',
  'updated_at': None},
 {'user_id': 5,
  'user_name': 'nikstha',
  'email': 'nikshres52@gmail.com',
  'role': 'user',
  'created_at': '2024-11-07T20:22:48.435697-05:00',
  'updated_at': None},
 {'user_id': 6,
  'user_name': 'User19',
  'email': 'user19@example.com',
  'role': 'user',
  'created_at':

In [19]:
import pandas as pd
import psycopg2

# Database credentials
username="rec_user"
password="admin1234"
host="localhost"
database="recommendation_system"
port="5432"


# Use psycopg2 to connect
conn = psycopg2.connect(
    dbname=database,
    user=username,
    password=password,
    host=host,
    port=port
)
cur = conn.cursor()

# Define the mapping dictionary
user_id_mapping = {
    19: 6,
    77: 7,
    75: 8
}

# Step 3: Filter the DataFrame to include only the user_ids in the mapping (optional)
mapped_df = df[df['user_id'].isin(user_id_mapping.keys())].copy()

# Step 4: Apply the mapping to the 'user_id' column
mapped_df['user_id'] = mapped_df['user_id'].map(user_id_mapping)


# Step 3: Insert Data into 'ratings' Table
try:
    # Prepare the SQL INSERT statement
    insert_query = """
    INSERT INTO ratings (movie_id, user_id, rating)
    VALUES (%s, %s, %s)
    """

    # Convert DataFrame to a list of tuples
    data_to_insert = list(mapped_df.itertuples(index=False, name=None))

    # Execute the batch insertion
    cur.executemany(insert_query, data_to_insert)

    # Commit the transaction
    conn.commit()
    print("Data inserted successfully.")

except Exception as e:
    print(f"An error occurred: {e}")
    conn.rollback()
finally:
    # Close the cursor and connection
    cur.close()
    conn.close()


Data inserted successfully.


In [17]:
df.head()

,item_id,user_id,rating
0,1,19,5
1,6,19,3
2,10,19,3
3,16,19,3
4,17,19,3
